## Importing CelebDataset from Kaggle

in uploading section uploading .json API file dowloaded from kaggle personal account



In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"manindersingh12","key":"c00d1bfcc3698b9650b93e98b9809900"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download jessicali9530/celeba-dataset
!unzip -d dataset celeba-dataset.zip

In [ ]:
#Importing necessary files
#note: eager execution command is necessary as it was showing some error in my runtime

import numpy as np
import os
import tensorflow as tf; tf.compat.v1.disable_eager_execution()  #importAnt for executing VAE
from tensorflow.keras.layers import Input,Dense,Flatten,Lambda,Reshape,Conv2D,Conv2DTranspose,LeakyReLU,Activation
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
import warnings

warnings.filterwarnings("ignore")
np.random.seed(25)  
tf.executing_eagerly()

False

In [ ]:
import PIL
PIL.Image.open('/content/dataset/img_align_celeba/img_align_celeba/000002.jpg').size

(178, 218)

In [ ]:
import tensorflow as tf
import keras
dataset_directory = "/content/dataset/img_align_celeba/img_align_celeba"

## inputing image dataset as such from folders
# deviding into train and validation set accordingly which can be used to generate new images and test for reconstructed images

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    label_mode=None,
    image_size=(128, 128),
    batch_size=256,
    validation_split=0.2,
    subset="training",
    seed=232

)
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     dataset_directory,
#     label_mode=None,
#     image_size=(128, 128),
#     batch_size=32,
#     validation_split=0.2,
#     subset="validation",
#     seed=232

# )


train_ds = train_ds.map(lambda x: x / 255.0)



Found 202599 files belonging to 1 classes.
Using 101300 files for training.
Found 202599 files belonging to 1 classes.
Using 101299 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


In [ ]:
def parse_image(data):
    img = tf.io.read_file(data_dir + "/"+ data)
    img = tf.image.decode_jpeg(img, channels=3)
    
    shapes = tf.shape(img)
    h, w = shapes[-3], shapes[-2]
    small = tf.minimum(h, w)
    
    img = tf.image.convert_image_dtype(img, tf.float32) 
    img = tf.image.resize_with_crop_or_pad(img, small, small) 
    img = tf.image.resize(img, [image_size, image_size], antialias=True)
    return img

data_dir = '/content/dataset/img_align_celeba/img_align_celeba' 
data = os.listdir(data_dir)
# image_size = 128
# batch_size = 128
# AUTOTUNE = tf.data.AUTOTUNE



list_ds = tf.data.Dataset.from_tensor_slices(data)
# images_ds = list_ds.map(parse_image, num_parallel_calls=AUTOTUNE)
# images_ds = images_ds.cache().shuffle(len(data)).batch(batch_size).prefetch(buffer_size=AUTOTUNE)


In [ ]:
#CONSTRucting encoder
image_width = 128
image_height = 128
channels = 3
latent_space_dim = 200 # dimension of the latent space

encoder_input = Input(shape=(image_height,image_width,channels),name='Enoder_Input')
x = Conv2D(filters=32,kernel_size=3,strides=1,padding='same',
                      name='EncoderConv1')(encoder_input)
x = LeakyReLU(name='LeakyReLU_1')(x)
x = Conv2D(filters=64,kernel_size=3,strides=2,padding='same',
                      name='EncoderConv2')(x)
x = LeakyReLU(name='LeakyReLU_2')(x)
x = Conv2D(filters=64,kernel_size=3,strides=2,padding='same',
                      name='EncoderConv3')(x)
x = LeakyReLU(name='LeakyReLU_3')(x)
x = Conv2D(filters=64,kernel_size=3,strides=1,padding='same',
                      name='EncoderConv4')(x)
x = LeakyReLU(name='LeakyReLU_4')(x)

# Storing Shape before flattening as it will be usefull in case of encoder to 
# convert latent space input back to this dimension to perform further operations
# of decoding
shape_before_flattening = K.int_shape(x)[1:] 
x = Flatten()(x)

######
###### Instead Of directly Connecting it to output of latent space, Connecting 
## it to mu and sigma layer to perform certain operation

## LATENT SPACE DIMENSION IS USED HERE
mu = Dense(latent_space_dim)(x)
sigma = Dense(latent_space_dim)(x)


#Model that takes input image and gives relevant mu and sigma values
encoder_mu_log_sigma = Model(encoder_input,(mu,sigma)) # Not a necessary step as such

# Creating Function to be used in a model Throug LAMBDA LAYER
def sampling(args):
  mu, sigma = args
  epsilon = K.random_normal(K.shape(mu),mean = 0.,stddev=1.0)
  return mu + K.exp(sigma / 2) * epsilon

encoder_output = Lambda(sampling,name='encoder_output')([mu,sigma])

encoder = Model(encoder_input,encoder_output)

In [ ]:
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enoder_Input (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 EncoderConv1 (Conv2D)          (None, 128, 128, 32  896         ['Enoder_Input[0][0]']           
                                )                                                                 
                                                                                                  
 LeakyReLU_1 (LeakyReLU)        (None, 128, 128, 32  0           ['EncoderConv1[0][0]']           
                                )                                                           

In [ ]:

decoder_input = Input(shape=(latent_space_dim,),name='decoder_input')

x = Dense(np.prod(shape_before_flattening))(decoder_input)# np.prod() multiplies all th eelemnt to give same no. of nodes as encoder
x = Reshape(shape_before_flattening)(x)
x = Conv2DTranspose(filters=64,padding='same',kernel_size=3,strides=1
                    ,name='ConveT_Decoder1')(x)
x = LeakyReLU()(x)
x = Conv2DTranspose(filters=64,padding='same',strides=2,kernel_size=3
                    ,name='ConveT_Decoder2')(x)
x = LeakyReLU()(x)
x = Conv2DTranspose(filters=32,padding='same',strides=2,kernel_size=3
                    ,name='ConveT_Decoder3')(x)
x = LeakyReLU()(x)
x = Conv2DTranspose(filters=1,padding='same',strides=1,
                    kernel_size = 3,name='ConveT_decoder4')(x)
Decoder_output = Activation('sigmoid')(x)

decoder_model = Model(decoder_input,Decoder_output)



In [ ]:
shape_before_flattening

(32, 32, 64)

In [ ]:
np.prod(shape_before_flattening)

65536

In [ ]:
decoder_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 200)]             0         
                                                                 
 dense_2 (Dense)             (None, 65536)             13172736  
                                                                 
 reshape (Reshape)           (None, 32, 32, 64)        0         
                                                                 
 ConveT_Decoder1 (Conv2DTran  (None, 32, 32, 64)       36928     
 spose)                                                          
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 ConveT_Decoder2 (Conv2DTran  (None, 64, 64, 64)       36928     
 spose)                                                    

In [ ]:

model_input = encoder_input  #taking encoder input as inpjut
model_output = decoder_model(encoder_output)  # taking output from decoder model 
                                  # which takes output from encoder output as input
model = Model(model_input,model_output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enoder_Input (InputLayer)      [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 EncoderConv1 (Conv2D)          (None, 128, 128, 32  896         ['Enoder_Input[0][0]']           
                                )                                                                 
                                                                                                  
 LeakyReLU_1 (LeakyReLU)        (None, 128, 128, 32  0           ['EncoderConv1[0][0]']           
                                )                                                           

Creating loss fujnction

In [ ]:

def vae_r_loss(y_true, y_pred):
  r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
  return 1000 * r_loss

def vae_kl_loss(y_true, y_pred):
  kl_loss =  -0.5 * K.sum(1 + sigma - K.square(mu) - K.exp(sigma), axis = 1)
  return kl_loss

def vae_loss(y_true, y_pred):
  r_loss = vae_r_loss(y_true, y_pred)
  kl_loss = vae_kl_loss(y_true, y_pred)
  return  r_loss + kl_loss


model.compile(optimizer='adam',loss = vae_loss,  metrics = [vae_r_loss, vae_kl_loss])

In [ ]:
# training the model
history = model.fit(train_ds,epochs=20)

In [ ]:
encoded = model.predict(val_ds)


# New image recreation

This can be used to compare generated images also but here our goal is to Recreate new Images

In [ ]:
n_to_show = 20
z_new = np.random.normal(size=(n_to_show,latent_space_dim))

reconstruction = decoder_model.predict(np.array(z_new))

fig = plt.figure(figsize=(18,5))
fig.subplots_adjust(hspace=0.4,wspace = 0.4)
for i in range(n_to_show):
  ax = fig.add_subplot(3,10,i+1)
  ax.imshow(np.squeeze(reconstruction[i,:,:,:]))
  ax.axis('off')

## One of the potential use of latent space conversion is that it is used to transform image features into vectors such as we can change few vectors to change the features of images such as adding sunglasses to image, adding or removing smile to image and many things more.


---



## we can also use VAE to morphing one image into another using formaula as below


```
 z_new = z_A *(1-alpha) + z_B * alpha

Here, alpha is a number between 0 and 1 that determines how far along the line we are away from point A

```

We takes two images, encoding them and than decoding poiints  from latent space in a straight line between these two points in regular interval thus obtaining smooth transition between images

